# No SQL Databases - MongoDb

In deze bijhorende notebook gaan we een voorbeeld uitwerken van een Document Based NoSQL database.
Hiervoor maken we gebruik van MongoDb, wat de mogelijkheid aanbied om het lokaal te draaien.
Deze applicatie voor het beheer van een NoSQL database is reeds geinstalleerd en kan gestart worden door het volgende commando in een terminal uit te voeren

    mongod
    
Dit commando start de MongoDb Server. 
Zolang deze applicatie draait kan je met MongoDb connecteren via een shell (niet geinstalleerd) of een api zoals pymongo (is geinstalleerd).

In MongoDb begin je met een te connecteren met een bepaalde database.
Dit doe je door een host en poort te kiezen en de naam van een bepaalde database.
Dit is analoog met hoe je een SQL-database aanspreekt.

MongoDb is een document-based NoSqlDatabase wat betekend dat een database bestaat uit een reeks collecties die elk een aantal documenten bevatten.

In de code hieronder connecteren we met een lokale database "les" waarin we twee collecties gaan gebruiken, namelijk "vakken" en "studenten". 
Deze collecties zijn conceptueel analoog aan de tabellen in een SQL-database.

In [1]:
import pymongo

In [4]:
client = pymongo.MongoClient("localhost", 27017)

#client.drop_database("herkansing")
#client.drop_database("les")
client.list_database_names()

database = client.les
coll_studenten = database.studenten
coll_vakken = database.vakken

print(client.list_database_names())
print(database.list_collection_names())

['admin', 'config', 'local']
[]


Bij bovenstaande code is er echter nog een belangrijke opmerking:
**De database en collecties worden lazy aangemaakt**. 
Dit houdt in dat ze maar aangemaakt worden wanneer ze effectief gebruikt worden (dus wanneer er een document toegevoegd wordt).
Bovenstaande code gaat op dit moment nog geen database en collecties aanmaken.

De documenten in MongoDb kunnen voorgesteld worden als Json formaat. 
In python kunnen dictionaries gebruikt worden om deze documenten voor te stellen, bvb voor een de drie vakken van dit keuzetraject:

In [5]:
datascience = {
    "naam": "Data Science",
    "studiepunten": 5,
    "semester": 1
}

bigdata = {
    "naam": "Big Data",
    "studiepunten": 5,
    "semester": 2
}

machinelearning = {
    "naam": "Machine Learning",
    "studiepunten": 6,
    "semester": 1
}

Deze documenten kunnen toegevoegd worden aan de database door middel van volgende code.

In [7]:
coll_vakken.insert_one(datascience)

DuplicateKeyError: E11000 duplicate key error collection: les.vakken index: _id_ dup key: { _id: ObjectId('627a2776b4f027a55b6b7f0d') }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('627a2776b4f027a55b6b7f0d')}, 'errmsg': "E11000 duplicate key error collection: les.vakken index: _id_ dup key: { _id: ObjectId('627a2776b4f027a55b6b7f0d') }"}

In [8]:
coll_vakken.insert_many([bigdata, machinelearning])

Dat deze documenten goed zijn toegevoegd en de nodige databases en collecties aangemaakt zijn kan gecontroleerd worden op de volgende manier:

In [15]:
#results = coll_vakken.find({})
results = coll_vakken.find({"naam" : "Data Science"})



from pprint import pprint

for result in results:
    pprint(result)

{'_id': ObjectId('627a2776b4f027a55b6b7f0d'),
 'naam': 'Data Science',
 'semester': 1,
 'studiepunten': 5}


Om de vakken toe te voegen hebben we documenten 1 voor 1 toegevoegd.
Een andere manier is om met een rij van dictionaries te werken om meerdere documenten tegelijkertijd toe te voegen. 
Dit kan bijvoorbeeld als volgt gedaan worden:

In [16]:
import datetime

students = [{
    "studentennummer": 202001546,
    "naam": "Andy Weir",
    "vakken": [{"naam" : "Data Science", "score": 8}, 
               {"naam" : "Big Data", "score": 10}, 
               {"naam" : "Machine Learning", "score": 12}],
    "geboortedatum": datetime.datetime(2000, 4, 24)
},{
    "studentennummer": 202001548,
    "naam": "Albus Dumbledore",
    "vakken": [{"naam" : "Data Science", "score": 14}, 
               {"naam" : "Big Data", "score": 16}, 
               {"naam" : "Machine Learning", "score": 15}],
    "geboortedatum": datetime.datetime(1800, 4, 24)
},{
    "studentennummer": 202001556,
    "naam": "Frodo Baggings",
    "vakken": [{"naam" : "Data Science", "score": 3}, 
               {"naam" : "Big Data", "score": 5}, 
               {"naam" : "Machine Learning", "score": 4}],
    "geboortedatum": datetime.datetime(1960, 4, 24)
}]

coll_studenten.insert_many(students)

In [17]:
for doc in coll_studenten.find():
    pprint(doc)

{'_id': ObjectId('627a290cb4f027a55b6b7f10'),
 'geboortedatum': datetime.datetime(2000, 4, 24, 0, 0),
 'naam': 'Andy Weir',
 'studentennummer': 202001546,
 'vakken': [{'naam': 'Data Science', 'score': 8},
            {'naam': 'Big Data', 'score': 10},
            {'naam': 'Machine Learning', 'score': 12}]}
{'_id': ObjectId('627a290cb4f027a55b6b7f11'),
 'geboortedatum': datetime.datetime(1800, 4, 24, 0, 0),
 'naam': 'Albus Dumbledore',
 'studentennummer': 202001548,
 'vakken': [{'naam': 'Data Science', 'score': 14},
            {'naam': 'Big Data', 'score': 16},
            {'naam': 'Machine Learning', 'score': 15}]}
{'_id': ObjectId('627a290cb4f027a55b6b7f12'),
 'geboortedatum': datetime.datetime(1960, 4, 24, 0, 0),
 'naam': 'Frodo Baggings',
 'studentennummer': 202001556,
 'vakken': [{'naam': 'Data Science', 'score': 3},
            {'naam': 'Big Data', 'score': 5},
            {'naam': 'Machine Learning', 'score': 4}]}


Om complexere queries uit te voeren moet er gebruik gemaakt worden van de [aggregate functie](https://pymongo.readthedocs.io/en/stable/examples/aggregation.html) waarbij je een stappenplan kan meegeven om een eindresultaat te bekomen.
Meer informatie over alles wat je kan doen met deze aggregate functie kan je vinden in de documentatie van [MongoDb](https://docs.mongodb.com/manual/aggregation/).
Bekijk hiervan zeker de documentatie over [de werking van de pipelines](https://docs.mongodb.com/manual/core/aggregation-pipeline/#std-label-aggregation-pipeline) en de [operators](https://docs.mongodb.com/manual/reference/operator/aggregation/#std-label-aggregation-expression-operators) die je kan gebruiken bij het opstellen van deze pipeline
Nu gaan we een aantal zaken proberen te bereken uit deze data, namelijk:
* Hoeveel vakken zijn er voor elk verschillend aantal studiepunten?
 * Correcte antwoord: 5 studiepunten -> 2 vakken, 6 studiepunten -> 1 vak
* Hoeveel studenten heeft elk vak?
* Voor welke vakken is elke student geslaagd?

**Updaten**

Met behulp van de find_one_and_update functie kunnen we gegevens wijzigen.
In de code hieronder gaan we 
* de naam van het vak Data Science wijzigen naar data (en terug)
* het studentennummer met eentje verhogen van Andy Weir
* de score van Andy Weir voor het vak Big Data veranderen naar 20

**Verwijderen**

Naast het updaten is het ook mogelijk om verscheidene elementen te verwijderen.
Dit kan aan de hand van een query of door de gewenste collections/databasen te verwijderen.
De code hiervoor is als volgt: